# Mejorando el Ambiente de Prueba

Para no tener que buildiar la imagen de *fastapi-app-image* cada vez que hay un cambio, unicamente el  contenedor de *tuTiendaDB* va estar corriendo con la BD. *main.py* se correra con *uvicorn*

```sh
uvicorn main:app --reload
```

En el *main.py* se cambia el *connection string*, recordemos el servicio de *mongo* esta expuesto en el puerto *27017* en la maquina local. 

`MONGO_DETAILS = "mongodb://localhost:27017/"`

## Creando una base de datos local

Simplemente para ir practicando consultas con parametros de ruta, y parametros de consulta(query parameters).

Aqui la primera implementacion, funciona desde `/clients`:

```py
@app.get("/client/{client_id}", tags=['client'])
async def get_user_by_id(client_id: int):

    return JSONResponse(content={"message": f"your client id: {client_id}"})
```

Pero es hora de hacerla mas robusta. 

### Usando Path 

Path es una función especial de FastAPI que usas para declarar y añadir validación y metadatos a los parámetros de ruta de tus funciones de endpoint.


```py
@app.get("/client/{client_id}", tags=['client'])
async def get_client_by_id(client_id: int = Path(..., 
                                                 gt=0, 
                                                 description="ID del cliente a buscar")):

    return JSONResponse(content={"message": f"your client id: {client_id}"})
```


- `client_id: int`: Esto le dice a Python (y a FastAPI/Pydantic) que `client_id` es una variable que debería ser un número entero (int). Esto se conoce como anotación de tipo (type hint).

- Path(...): Esta es la función clave de FastAPI. Cuando la usas, le estás diciendo a FastAPI que `client_id` no es un parámetro de consulta (como `?client_id=123`), sino un parámetro que se espera que esté en la URL de la ruta (como `/client/123`).

#### Para que se usa PATH


- El `...` (Ellipsis) dentro de Path() le dice a FastAPI que este parámetro de ruta es obligatorio. Si no se proporciona en la URL, FastAPI generará automáticamente un error.

- Añadir Validación de Datos:Puedes pasar argumentos a Path() para aplicar reglas de validación al valor del parámetro.`gt=0`: En tu ejemplo, esto significa "mayor que cero". FastAPI validará automáticamente que el client_id sea un número entero y que sea estrictamente mayor que cero

### Añadiendo un Codigo de Estado por Defecto

```py
@app.get("/client/{client_id}", tags=['client'], status_code=200)
async def get_client_by_id(client_id: int = Path(..., 
                                                 gt=0, 
                                                 description="ID del cliente a buscar")):

    return JSONResponse(content={"message": f"your client id: {client_id}"})
```


El código de estado `200` *OK* es el código HTTP estándar para una respuesta exitosa donde los datos solicitados se han devuelto correctamente. Al incluirlo en el decorador:

- Claridad y Consistencia: Se establece la expectativa de que, si todo sale bien, la API responderá con un `200`. Esto hace que tu API sea más predecible para quienes la consumen.

- Documentación Implícita: Aunque FastAPI ya lo infiere por defecto para `GET requests`, especificarlo explícitamente refuerza la intención y aparece en la documentación de Swagger UI.

- Buen Punto de Partida: Sirve como el estado por defecto para el "camino feliz" (cuando todo funciona como se espera). Luego, puedes sobrescribir este comportamiento para casos específicos, como levantar un `HTTPException` con un `404` *Not Found* si el cliente no existe, como lo harías en una implementación más robusta.

### Mejorando la Funcion

```py
# Path Parameter para encontrar un cliente por su id 
@app.get("/client/{client_id_to_find}", tags=['client'], status_code=200)
async def get_client_by_id(client_id_to_find: int = Path(..., 
                                                 gt=0, 
                                                 description="ID del cliente a buscar")):
    
    """
    Busca un cliente en DB y devuelve todos sus datos si es encontrado. 👈
    Si el cliente no existe, retorna un error 404 Not Found.
    """
    found_client_list = []

    for client in clients:
        if client['id']==client_id_to_find:   
            found_client_list.append(client)
            break 

    if not found_client_list:
        message = f"cliente con id {client_id_to_find} no existe"
        raise HTTPException(status_code=404, detail={"message":message}) 👈

    else:
        return JSONResponse(status_code=200, content=found_client_list[0]) 👈
```

Esta funcion mejora aspectos clave:

- incluye `dotstrings` para ir documentando el proyecto automaticamente
- Si el cliente no se encuentra, retorna un codigo de estado *404* y un mensaje
- Si lo encuentra envia todos los datos del cliente.